# Vamos a intentar reconstruir la lista del S&P500 a partir de la página de wikipedia:

Necesitaremos la lista de miembros en cada momento para mas de una de las estrategias.

In [1]:
import requests

parameters = {'action' : 'query', 
              'rvprop' : 'content',
              'format' : 'json',
              'prop' : 'revisions',
              'titles' : 'List_of_S&P_500_companies'}
r = requests.get('https://en.wikipedia.org/w/api.php',
                 params=parameters)

In [2]:
import json

jresp = json.loads(r.text)

jresp['query']['pages']['2676045']['revisions']

[{u'*': u'The \'\'\'[[S&P 500]]\'\'\' [[stock market index]], maintained by [[S&P Dow Jones Indices]], comprises \'\'\'505\'\'\' [[common stock]]<nowiki/>s issued by \'\'\'500\'\'\' [[market capitalization|mid and large-cap]] companies and traded on American stock exchanges, and covers about 75 percent of the American equity market by capitalization. The index is weighted by free-float market capitalization, so more valuable companies account for relatively more of the index. The index constituents and the constituent weights are updated and checked on regularly using rules published by S&P Dow Jones Indices. Although the index is called the S&P "500", the index contains 505 stocks because it includes two share classes of stock from 5 of its component companies.<ref>{{cite news|url=http://americasmarkets.usatoday.com/2014/07/30/the-sp-500-is-becoming-the-sp-502/ |title=The S&P 500 is becoming the S&P 502 . |publisher=USA Today |date=2014-07-30 |accessdate=2014-12-02}}</ref><ref>{{cite 

In [3]:
import urllib2
from lxml.html import parse
from IPython.core.display import HTML

response = urllib2.urlopen('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#page = response.read() # This empties the response!

tree = parse(response)

In [4]:
tables = tree.getroot().findall('.//table')

In [5]:
current_members = tables[0]

In [6]:
current_members.xpath('//tr')[0].text_content().split('\n')


['',
 'Ticker symbol',
 'Security',
 'SEC filings',
 'GICS Sector',
 'GICS Sub Industry',
 'Address of Headquarters',
 'Date first added',
 'CIK',
 '']

In [7]:
raw_rows = current_members.xpath('//tr')

header = raw_rows[0].text_content().split('\n')[1:-1]
rows = []

for row in raw_rows[1:]:
    elems = row.text_content().split('\n')[1:-1]
    rows.append(elems)
    

In [8]:
import pandas as pd

df = pd.DataFrame(rows)
df.columns=header
df = df.set_index('Ticker symbol')

In [9]:
df.head()

,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
Ticker symbol,,,,,,,
MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",,0000066740
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",1964-03-31,0000001800
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152
ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373
ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,0000718877


In [10]:
changes = tables[1]

In [11]:
changes.xpath('//tr')[0].text_content() # WTF? gets me the header of the first table

'\nTicker symbol\nSecurity\nSEC filings\nGICS Sector\nGICS Sub Industry\nAddress of Headquarters\nDate first added\nCIK\n'

In [12]:
# Lo hacemos a pelo



header = changes.text_content().split('\n')[6:14]
header[2] = 'added_' + header[2]
header[3] = 'added_' + header[3]
header[4] = 'removed_' + header[4]
header[5] = 'removed_' + header[5]

body = changes.text_content().split('\n')[14:]


In [13]:
lines = []
for i in range(0, len(body), 8):
    line = body[i + 1 : i + 7]
    lines.append(line)
    
changes_df = pd.DataFrame(lines)
changes_df.columns = header[1:7]
changes_df

,Date,added_Ticker,added_Security,removed_Ticker,removed_Security,Reason
0,"October 3, 2016",COTY,"Coty, Inc",DO,Diamond Offshore Drilling Inc,COTY replaces DO[9]
1,"September 23, 2016",COO,The Cooper Companies,HOT,Starwood Hotels & Resorts Worldwide Inc,MAR acquires HOT[10]
2,"September 8, 2016",CHTR,Charter Communications,EMC,EMC Corporation,Dell acquires EMC[11]
3,"September 6, 2016",MTD,Mettler Toledo,JCI,Johnson Controls Inc,TYC acquires JCI (and becomes new JCI)[12]
4,"July 5, 2016",FTV,Fortive Corp,CPGX,Columbia Pipeline Group Inc,CPGX acquired by TRP[13]
5,"July 1, 2016",LNT,Alliant Energy Corp,GAS,AGL Resources,GAS acquired by SO[14]
6,"July 1, 2016",ALB,Albemarle Corp,TE,TECO Energy,TE acquired by EMA[15]
7,"June 22, 2016",FBHS,Fortune Brands Home & Security,CVC,Cablevision Systems,CVC acquired by Altice NV[16]
8,"June 3, 2016",TDG,TransDigm Group,BXLT,Baxalta Inc,SHPG acquiring BXLT[17]
9,"May 31, 2016",AJG,Arthur J. Gallagher & Co.,CCE,Coca-Cola Enterprises,CCE merging with European bottlers[18]
